In [13]:
import warnings

warnings.filterwarnings('ignore')

import os

from tools.modeling.timeserieshyperparamtuner import TimeSeriesHyperParamTuner
from tools.modeling.timeseriesmodeler import TimeSeriesModeler

from nba.players import get_all_players_by_season, get_players_game_log
from nba.common import add_game_log_lag, add_game_log_last_5
import nba.constants as c

from xgboost import XGBRegressor
from hyperopt import hp

import numpy as np
import pandas as pd

import shap


ROOT_DIR = os.getcwd()[:-8] # verify this points to the root directory of the project

season = c.SEASON_2022_2023

ROOT_DIR

'/Users/rmcdevi2/Projects/personal/nba-analysis/'

In [14]:
# Get all players game logs for the top 100 scorers of the season
df = get_all_players_by_season(season)
df = df.loc[df[c.GP] >= 30]
df = df.sort_values(by=c.PTS, ascending=False)
df[c.PTS].describe()
# player_ids = df[c.PLAYER_ID].head(100).tolist()
# logs = get_players_game_log(player_ids, season)

count    405.000000
mean      10.724938
std        6.782676
min        1.300000
25%        5.600000
50%        9.000000
75%       14.100000
max       33.100000
Name: PTS, dtype: float64

In [15]:
# logs.to_csv(ROOT_DIR + '/data/game_logs/{season}/top_100_scorers_{season}_game_log.csv'.format(season=season), index=False)